In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, f_oneway, kruskal, chisquare
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv('StudentsPerformance.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


In [3]:
df['average score'] = df[['math score', 'reading score', 'writing score']].mean(axis=1)
df[['math score', 'reading score', 'writing score', 'average score']].head()


,math score,reading score,writing score,average score
0,72,72,74,72.666667
1,69,90,88,82.333333
2,90,95,93,92.666667
3,47,57,44,49.333333
4,76,78,75,76.333333


In [4]:
high_school = df[df['parental level of education'] == 'some high school']['average score'].dropna()
bachelor_degree = df[df['parental level of education'] == 'bachelor\'s degree']['average score'].dropna()
master_degree = df[df['parental level of education'] == 'master\'s degree']['average score'].dropna()
associate_degree = df[df['parental level of education'] == 'associate\'s degree']['average score'].dropna()
high_school_grad = df[df['parental level of education'] == 'high school']['average score'].dropna()


In [5]:
groups = [high_school, bachelor_degree, master_degree, associate_degree, high_school_grad]
group_names = ['some high school', 'bachelor\'s degree', 'master\'s degree', 'associate\'s degree', 'high school']

In [6]:
def check_normality(groups, group_names):
    normal_groups = []
    non_normal_groups = []

    for group, name in zip(groups, group_names):
        if len(group) > 1:
            stat, p_val = shapiro(group)
            print(f"Shapiro-Wilk test для группы '{name}': p-value = {p_val:.3f}")

            if p_val < 0.05:
                print(f"❌ Распределение **не нормальное** для группы '{name}'")
                non_normal_groups.append(group)
            else:
                print(f"✅ Распределение **нормальное** для группы '{name}'")
                normal_groups.append(group)
        else:
            print(f"❌ Недостаточно данных для группы '{name}'")

    return normal_groups, non_normal_groups


normal_groups, non_normal_groups = check_normality(groups, group_names)

Shapiro-Wilk test для группы 'some high school': p-value = 0.009
❌ Распределение **не нормальное** для группы 'some high school'
Shapiro-Wilk test для группы 'bachelor's degree': p-value = 0.340
✅ Распределение **нормальное** для группы 'bachelor's degree'
Shapiro-Wilk test для группы 'master's degree': p-value = 0.230
✅ Распределение **нормальное** для группы 'master's degree'
Shapiro-Wilk test для группы 'associate's degree': p-value = 0.110
✅ Распределение **нормальное** для группы 'associate's degree'
Shapiro-Wilk test для группы 'high school': p-value = 0.105
✅ Распределение **нормальное** для группы 'high school'


In [7]:
if normal_groups:
    print("\nПроверка с помощью ANOVA для нормальных распределений:")
f_oneway(*normal_groups)



Проверка с помощью ANOVA для нормальных распределений:


F_onewayResult(statistic=np.float64(15.829499650328307), pvalue=np.float64(6.519462218264812e-10))

In [11]:
f_stat, p_val = f_oneway(*normal_groups)
print(f"ANOVA: F-statistic = {f_stat:.3f}, P-value = {p_val:.3f}")
if p_val < 0.05:
    print("✅ Отклоняем H₀: различия между группами значимы.")
else:
    print("❌ Не отклоняем H₀: различия между группами незначимы.")

ANOVA: F-statistic = 15.829, P-value = 0.000
✅ Отклоняем H₀: различия между группами значимы.


In [20]:
groups = [high_school, bachelor_degree, master_degree, associate_degree, high_school_grad]
group_names = ['some high school', 'bachelor\'s degree', 'master\'s degree', 'associate\'s degree', 'high school']

if non_normal_groups:
    print("\nПроверка с помощью Краскела-Уоллиса для ненормальных распределений:")

    if len(non_normal_groups) > 1:
        h_stat, p_val_kruskal = kruskal(*non_normal_groups)
        print(f"Краскел-Уоллис: H-statistic = {h_stat:.3f}, P-value = {p_val_kruskal:.3f}")

        if p_val_kruskal < 0.05:
            print("✅ Отклоняем H₀: различия между группами значимы.")
        else:
            print("❌ Не отклоняем H₀: различия между группами незначимы.")
    else:
        print("❌ Недостаточно данных для применения Краскела-Уоллиса.")


Проверка с помощью Краскела-Уоллиса для ненормальных распределений:
❌ Недостаточно данных для применения Краскела-Уоллиса.


In [21]:
def categorize(score):
    if score < 60:
        return 'низкий'
    elif score <= 80:
        return 'средний'
    else:
        return 'высокий'

df['score_category'] = df['average score'].apply(categorize)
subset = df[df['parental level of education'] == 'high school']

In [22]:
counts = pd.crosstab(index=subset['parental level of education'], columns=subset['score_category'])

print("\nСводная таблица (наблюдаемые значения):\n", counts)

observed = counts.values[0]
expected = [observed.sum() / len(observed)] * len(observed)
chisquare(observed, expected)


Сводная таблица (наблюдаемые значения):
 score_category               высокий  низкий  средний
parental level of education                          
high school                       18      76      102


Power_divergenceResult(statistic=np.float64(56.61224489795919), pvalue=np.float64(5.091048288480087e-13))

In [23]:
print("\nОжидаемые значения:\n", expected)
chi2_stat, p_value = chisquare(f_obs=observed, f_exp=expected)

print(f"\nχ²-статистика: {chi2_stat:.3f}")
print(f"P-value: {p_value:.3f}")

if p_value < 0.05:
    print("✅ Распределение категорий среднего балла НЕ является равномерным (есть статистическая значимость).")
else:
    print("❌ Распределение категорий среднего балла можно считать равномерным (нет статистической значимости).")


Ожидаемые значения:
 [np.float64(65.33333333333333), np.float64(65.33333333333333), np.float64(65.33333333333333)]

χ²-статистика: 56.612
P-value: 0.000
✅ Распределение категорий среднего балла НЕ является равномерным (есть статистическая значимость).


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       1000 non-null   object 
 1   race/ethnicity               1000 non-null   object 
 2   parental level of education  1000 non-null   object 
 3   lunch                        1000 non-null   object 
 4   test preparation course      1000 non-null   object 
 5   math score                   1000 non-null   int64  
 6   reading score                1000 non-null   int64  
 7   writing score                1000 non-null   int64  
 8   average score                1000 non-null   float64
dtypes: float64(1), int64(3), object(5)
memory usage: 70.4+ KB


In [12]:
grouped = df.groupby(['test preparation course', 'parental level of education']).size().reset_index(name='Count')
print(grouped)

   test preparation course parental level of education  Count
0                completed          associate's degree     82
1                completed           bachelor's degree     46
2                completed                 high school     56
3                completed             master's degree     20
4                completed                some college     77
5                completed            some high school     77
6                     none          associate's degree    140
7                     none           bachelor's degree     72
8                     none                 high school    140
9                     none             master's degree     39
10                    none                some college    149
11                    none            some high school    102


In [19]:
pivot_table = df.pivot_table(index='parental level of education',
                             columns='test preparation course',
                             aggfunc='size',
                             fill_value=0)
print(pivot_table)


test preparation course      completed  none
parental level of education                 
associate's degree                  82   140
bachelor's degree                   46    72
high school                         56   140
master's degree                     20    39
some college                        77   149
some high school                    77   102


In [14]:
observed = [82, 140, 46, 72, 56, 140, 20, 39, 77, 149, 77, 102]
expected = [71.55, 150.45, 45.55, 72.45, 56.91, 139.09, 26.55, 32.45, 59.73, 166.27, 78.73, 100.27]

chi2_stat, p_val = chisquare(observed, expected)

print(f"Chi-squared statistic: {chi2_stat:.3f}")
print(f"P-value: {p_val:.3f}")

if p_val < 0.05:
    print("✅ Отклоняем H₀: существует зависимость между курсом подготовки и уровнем образования родителей.")
else:
    print("❌ Не отклоняем H₀: зависимости между курсом подготовки и уровнем образования родителей нет.")


Chi-squared statistic: 12.073
P-value: 0.358
❌ Не отклоняем H₀: зависимости между курсом подготовки и уровнем образования родителей нет.
